In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()


def connect_to_db() -> psycopg2.extensions.connection:
    return psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
    )


def create_backup():
    conn = connect_to_db()
    cur = conn.cursor()

    try:
        # Drop existing backup table if it exists
        print("Lösche existierendes Backup falls vorhanden...")
        cur.execute(
            """
            DROP TABLE IF EXISTS dnb_records_subset_backup;
            """
        )

        # Backup erstellen
        print("Erstelle Backup...")
        cur.execute(
            """
            CREATE TABLE dnb_records_subset_backup AS 
            SELECT * FROM dnb_records_subset;
        """
        )

        # Überprüfe die Anzahl der Einträge
        cur.execute("SELECT COUNT(*) FROM dnb_records_subset")
        original_count = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM dnb_records_subset_backup")
        backup_count = cur.fetchone()[0]

        print(f"\nOriginal Einträge: {original_count}")
        print(f"Backup Einträge: {backup_count}")

        if original_count == backup_count:
            print("\nBackup erfolgreich erstellt!")

            # Zeige Verteilung in beiden Tabellen
            print("\nÜberprüfe DDC-Verteilung:")
            for table in ["dnb_records_subset", "dnb_records_subset_backup"]:
                cur.execute(
                    f"""
                    SELECT SUBSTRING(ddc FROM 1 FOR 1) as ddc_category, COUNT(*) as count
                    FROM {table}
                    GROUP BY ddc_category
                    ORDER BY ddc_category
                """
                )
                print(f"\n{table}:")
                for row in cur.fetchall():
                    print(f"DDC {row[0]}: {row[1]} Einträge")

            conn.commit()
        else:
            print("\nWARNUNG: Anzahl der Einträge stimmt nicht überein!")
            conn.rollback()

    except Exception as e:
        print(f"Fehler beim Backup: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()


if __name__ == "__main__":
    create_backup()